In [1]:
import os
import collections
import re
import json
import spacy
import pandas as pd

In [2]:
_CMAP_DIR = os.path.join("/home/stavros/GitHub/Review-Aspects-App/nlptools", "contractions.txt")
with open(_CMAP_DIR, "r") as file:
  _CMAP = json.load(file)


def expand_contractions(text, contraction_mapping=_CMAP):
  contractions_pattern = re.compile('({})'.format('|'.join(
      contraction_mapping.keys())), flags=re.IGNORECASE|re.DOTALL)

  def expand_match(contraction):
    match = contraction.group(0)
    first_char = match[0]
    if contraction_mapping.get(match):
      expanded_contraction = contraction_mapping.get(match)
    else:
      expanded_contraction = contraction_mapping.get(match.lower())
    expanded_contraction = first_char+expanded_contraction[1:]
    return expanded_contraction

  expanded_text = contractions_pattern.sub(expand_match, text)
  expanded_text = re.sub("'s", "", expanded_text)
  expanded_text = re.sub("'", "", expanded_text)
  return expanded_text

In [3]:
STOP_WORDS = set(
    """
a about above across after afterwards again against all almost alone along
already also although always am among amongst amount an and another any anyhow
anyone anything anyway anywhere are around as at
back be became because become becomes becoming been before beforehand behind
being below beside besides between beyond both bottom but by
call can cannot ca could
did do does doing done down due during
each eight either eleven else elsewhere empty enough even ever every
everyone everything everywhere except
few fifteen fifty first five for former formerly forty four from front full
further
get give go
had has have he hence her here hereafter hereby herein hereupon hers herself
him himself his how however hundred
i if in indeed into is it its itself
keep
last latter latterly least less
just
made make many may me meanwhile might mine more moreover most mostly move much
must my myself
name namely neither never nevertheless next nine no nobody none noone nor not
nothing now nowhere
of off often on once one only onto or other others otherwise our ours ourselves
out over own
part per perhaps please put
quite
rather re really regarding
same say see seem seemed seeming seems serious several she should show side
since six sixty so some somehow someone something sometime sometimes somewhere
still such
take ten than that the their them themselves then thence there thereafter
thereby therefore therein thereupon these they third this those though three
through throughout thru thus to together too top toward towards twelve twenty
two
under until up unless upon us used using
various very very via was we well were what whatever when whence whenever where
whereafter whereas whereby wherein whereupon wherever whether which while
whither who whoever whole whom whose why will with within without would
yet you your yours yourself yourselves
""".split()
)
len(STOP_WORDS)

305

In [7]:
hotel_name = "the_kresten_royal_villas"
n_reviews = 1748

data_dir = "/home/stavros/DATA/TripAdvisorReviews/{}".format(hotel_name)
data = pd.read_csv(os.path.join(data_dir, "{}_{}reviews.csv".format(hotel_name, n_reviews)))
print(data.shape)
print(data.columns)

published_year = data.publishedDate.apply(lambda x: x.split("-")[0])
published_year.value_counts()

(1748, 24)
Index(['id', 'absoluteUrl', 'createdDate', 'publishedDate', 'locationId',
       'originalLanguage', 'language', 'stayDate', 'tripType', 'helpfulVotes',
       'title', 'text', 'rating', 'additionalRatings', 'username', 'userId',
       'user_hometownId', 'user_hometownName', 'response_id',
       'response_publishedDate', 'response_language', 'response_username',
       'response_connectionToSubject', 'response_text'],
      dtype='object')


2015    274
2016    259
2017    221
2013    219
2018    195
2014    183
2019    182
2012    167
2011     48
Name: publishedDate, dtype: int64

In [8]:
nlp = spacy.load('en_core_web_sm')
expanded_text = data.text.apply(lambda x: expand_contractions(x))
data["spacy_text"] = list(nlp.pipe(expanded_text))

MemoryError: Unable to allocate array with shape (169714, 1024) and data type float32

In [ ]:
data.to_pickle(os.path.join(data_dir, "{}_{}reviews_spacy.pkl".format(hotel_name, n_reviews)))